In [1]:
import os
import torch
from torch.utils.data import Dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
import netCDF4 as nc
from skdownscale.pointwise_models import PointWiseDownscaler
from skdownscale.pointwise_models.bcsd import BcsdTemperature, BcsdPrecipitation
from sklearn.metrics import mean_squared_error


from skdownscale.pointwise_models import AnalogRegression, PureAnalog
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

/home/tedzed/anaconda3/lib/python3.7/site-packages/xarray/backends/cfgrib_.py:28: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  "Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. "


In [2]:
def crop_to_region(data, lat_range, lon_range):
    min_lat, max_lat = lat_range
    min_lon, max_lon = lon_range
    mask_lat = (data.lat >= min_lat) & (data.lat <= max_lat)
    mask_lon = (data.lon >= min_lon) & (data.lon <= max_lon)
    return data.where(mask_lat & mask_lon, drop =True)


def interpolate(data, lat_range, lon_range, steps, method="nearest"):
    lats = np.linspace(lat_range[0], lat_range[1], steps)
    lons = np.linspace(lon_range[0], lon_range[1], steps)
    interp_data = data.interp(lat=lats, lon=lons, method=method)
    interp_data = interp_data.fillna(0)
    return interp_data


from sklearn.metrics import mean_squared_error

def rms(y_test, out):
    rms = mean_squared_error(y_test, out, squared=False)
    return (rms)

def correlation(y_test, out): 
    import math
    y_t = y_train.to_numpy()
    y_t = y_t.reshape(365)

    o = out.to_numpy()
    o = o.reshape(365) 
    r, p = scipy.stats.pearsonr(y_t, o)
    return(math.sqrt(r))

#### Please replace with your directory. The data is on the drive mentioned on the github

In [3]:
file_hr = './Data/cpc-2000-precip.nc'
file_lr = './Data/erai-2000-precip.nc'

file_lr_pred = './Data/erai-2001-precip.nc'
file_hr_pred = "./Data/cpc-2001-precip.nc"


region = 'seus'
MONTHS = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul','aug', 'sep', 'oct', 'nov', 'dec']

#REGION_COORDS = {region:([38,48],[238,248])}
#REGION_COORDS = {'swus':([28,38],[268,278])}
#REGION_COORDS = {'neus':([35,45], [273,283])} 
REGION_COORDS = {'seus':([28,38], [268,278])} 



data_hr = xr.open_dataset(file_hr)
data_hr = data_hr.fillna(0)


data_lr = xr.open_dataset(file_lr)
data_lr = data_lr.fillna(0)
data_lr = data_lr.rename({'longitude':'lon', 'latitude':'lat'})
data_lr = data_lr.tp.resample(time='24H').sum('time')*1000




lat_range, lon_range = REGION_COORDS[region]
steps = 40


#Prediction Data
data_lr_p = xr.open_dataset(file_lr_pred)
data_lr_p = data_lr_p.fillna(0)
data_lr_p = data_lr_p.rename({'longitude':'lon', 'latitude':'lat'})
data_lr_p = data_lr_p.tp.resample(time='24H').sum('time')*1000

X_pred = interpolate(data_lr_p, lat_range, lon_range, steps=steps, method='nearest')
X_pred = X_pred.isel(lon=2,lat=2).to_dataframe()[['tp']] 
X_pred = X_pred.replace(0,1e-6)
X_pred = X_pred[:365]


y_train = interpolate(data_hr, lat_range, lon_range, steps=steps, method='nearest')
X_train = interpolate(data_lr, lat_range, lon_range, steps=steps, method='nearest')


y_train = y_train.isel(lon=2,lat=2).to_dataframe()[['precip']]
X_train = X_train.isel(lon=2,lat=2).to_dataframe()[['tp']]

y_train = y_train.replace(0,1e-7)
y_train = y_train[:365]

X_train = X_train[:365]
X_train = X_train.replace(0,1e-6)

/home/tedzed/anaconda3/lib/python3.7/site-packages/xarray/backends/plugins.py:68: RuntimeWarning: Engine 'cfgrib' loading failed:
Cannot find the ecCodes library
  warnings.warn(f"Engine {name!r} loading failed:\n{ex}", RuntimeWarning)


In [4]:
models = {
    'GARD: PureAnalog-best-1': PureAnalog(kind='best_analog', n_analogs=1),
    'GARD: PureAnalog-sample-10': PureAnalog(kind='sample_analogs', n_analogs=10),
    'GARD: PureAnalog-weight-10': PureAnalog(kind='weight_analogs', n_analogs=10),
    'GARD: PureAnalog-weight-100': PureAnalog(kind='weight_analogs', n_analogs=100), 
    'GARD: PureAnalog-mean-10': PureAnalog(kind='mean_analogs', n_analogs=10),
    'GARD: AnalogRegression-100': AnalogRegression(n_analogs=100),
    'Sklearn: RandomForestRegressor': RandomForestRegressor(random_state=0)
}


X = np.zeros((len(models),365, 40, 40))
for lat in range(X.shape[2]):
    for lon in range(X.shape[3]):
        
        X_pred = interpolate(data_lr_p, lat_range, lon_range, steps=steps, method='nearest')
        y_train = interpolate(data_hr, lat_range, lon_range, steps=steps, method='nearest')
        X_train = interpolate(data_lr, lat_range, lon_range, steps=steps, method='nearest')
        
        X_pred = X_pred.isel(lon=lon,lat=lat).to_dataframe()[['tp']] 
        X_pred = X_pred.replace(0,1e-6)
        X_pred = X_pred[:365]
        
        y_train = y_train.isel(lon=lon,lat=lat).to_dataframe()[['precip']]
        X_train = X_train.isel(lon=lon,lat=lat).to_dataframe()[['tp']]
        y_train = y_train[['precip']]
        X_train = X_train[['tp']]
        
        y_train = y_train.replace(0,1e-6)
        y_train = y_train[:365]
        
        X_train = X_train[:365]
        X_train = X_train.replace(0,1e-6)
       
        for index, (key, model) in enumerate(models.items()):
            model.fit(X_train, y_train)
            data = model.predict(X_pred)
            
           
            #print(data.shape)
            for t in range(X.shape[1]):
                X[index, t, lat, lon] = data[t] 
    

/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

...

/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/tedzed/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change t

In [5]:
np.save("GARD-seus", X)

In [6]:
data_hr = xr.open_dataset(file_hr_pred)
data_hr = data_hr.fillna(0)


Y = np.random.rand(365, 40, 40)
for lat in range(Y.shape[1]):
    for lon in range(Y.shape[2]):
        y_test = interpolate(data_hr, lat_range, lon_range, steps=steps, method='nearest')
        y_test = y_test.isel(lon=lon, lat=lat).to_dataframe().precip
        y_test = y_test.replace(0,1e-7)
        y_test = y_test[:365]
    
        for t in range(Y.shape[0]):
            Y[t, lat, lon] = y_test.values[t] 





In [7]:
np.max(Y)
np.save("Y_true-GARD-seus", Y)